This notebook demonstrates how to match metadata records in HTRC with metadata records from other sources.
In this example, we use strings of "author" and "title" for matching.
Here we use two datasets:(1) part of Hathifiles and (2) CMU Book Summary Dataset.

In [2]:
#Yuerong Hu
import pandas as pd
from fuzzywuzzy import fuzz

## Import and preprocess the datasets

1. Import the open access dataset downloaded from https://www.kaggle.com/applecrazy/cmu-book-summary-dataset

In [64]:
df=pd.read_csv('booksummaries.txt',sep='\t')

In [53]:
# take a look at the dataset

In [54]:
df.head()

620  /m/0hhy                                Animal Farm  \
0   843  /m/0k36                         A Clockwork Orange   
1   986  /m/0ldx                                 The Plague   
2  1756  /m/0sww  An Enquiry Concerning Human Understanding   
3  2080  /m/0wkt                       A Fire Upon the Deep   
4  2152  /m/0x5g             All Quiet on the Western Front   

          George Orwell  1945-08-17  \
0       Anthony Burgess        1962   
1          Albert Camus        1947   
2            David Hume         NaN   
3          Vernor Vinge         NaN   
4  Erich Maria Remarque  1929-01-29   

  {"/m/016lj8": "Roman \u00e0 clef", "/m/06nbt": "Satire", "/m/0dwly": "Children's literature", "/m/014dfn": "Speculative fiction", "/m/02xlf": "Fiction"}  \
0  {"/m/06n90": "Science Fiction", "/m/0l67h": "N...                                                                                                         
1  {"/m/02m4t": "Existentialism", "/m/02xlf": "Fi...                                                                                                         
2                                                NaN                                                                                                         
3  {"/m/03lrw": "Hard science fiction", "/m/06n90...                                                                                                         
4  {"/m/098tmk": "War novel", "/m/016lj8": "Roman...                                                                                                         

   Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a philosophy. The animals revolt and drive the drunken and irresponsible Mr Jones from the farm, renaming it "Animal Farm". They adopt Seven Commandments of Animal-ism, the most important of which is, "All animals are equal". Snowball attempts to teach the animals reading and writing; food is plentiful, and the farm runs smoothly. The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health. Napoleon takes the pups from the farm dogs and trains them privately. Napoleon and Snowball struggle for leadership. When Snowball announces his plans to build a windmill, Napoleon has his dogs chase Snowball away and declares himself leader. Napoleon enacts changes to the governance structure of the farm, replacing meetings with a committee of pigs, who will run the farm. Using a young pig named Squealer as a "mouthpiece", Napoleon claims credit for the windmill idea. The animals work harder with the promise of easier lives with the windmill. After a violent storm, the animals find the windmill annihilated. Napoleon and Squealer convince the animals that Snowball destroyed it, although the scorn of the neighbouring farmers suggests that its walls were too thin. Once Snowball becomes a scapegoat, Napoleon begins purging the farm with his dogs, killing animals he accuses of consorting with his old rival. He and the pigs abuse their power, imposing more control while reserving privileges for themselves and rewriting history, villainising Snowball and glorifying Napoleon. Squealer justifies every statement Napoleon makes, even the pigs' alteration of the Seven Commandments of Animalism to benefit themselves. 'Beasts of England' is replaced by an anthem glorifying Napoleon, who appears to be adopting the lifestyle of a man. The animals remain convinced that they are better off than they were when under Mr Jones. Squealer abuses the animals' poor memories and invents numbers to show their improvement. Mr Frederick, one of the neighbouring farmers, attacks the farm, using blasting powder to blow up the restored windmill. Though the animals win the battle, they do 

In [55]:
# We need to add column names to it and select rows 
# Add column names according to the readme file
# 1. Wikipedia article ID
# 2. Freebase ID
# 3. Book title
# 4. Author
# 5. Publication date
# 6. Book genres (Freebase ID:name tuples)
# 7. Plot summary
# Taking into consideration the matching later ,we use "title" and "author" instead of "Book title" and "Author"
df.columns = ['Wikipedia article ID','Freebase ID','title','author','ublication date','Book genres','Plot summary']
df.head()

,Wikipedia article ID,Freebase ID,title,author,ublication date,Book genres,Plot summary
0,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
1,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
2,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
3,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
4,2152,/m/0x5g,All Quiet on the Western Front,Erich Maria Remarque,1929-01-29,"{""/m/098tmk"": ""War novel"", ""/m/016lj8"": ""Roman...","The book tells the story of Paul Bäumer, a Ge..."


In [56]:
# We only need two columns
df = df[['title','author']]
df.head()

,title,author
0,A Clockwork Orange,Anthony Burgess
1,The Plague,Albert Camus
2,An Enquiry Concerning Human Understanding,David Hume
3,A Fire Upon the Deep,Vernor Vinge
4,All Quiet on the Western Front,Erich Maria Remarque


In [57]:
# import htrc metadata
df2=pd.read_csv('hathi_upd_20190801.txt',sep='\t')
df2.head()

,mdp.39015018415946,deny,ic,000000001,v.5,MIU,000000001.1,2779601,Unnamed: 8,Unnamed: 9,...,9999,cs,slo,BK,MIU.1,umich,umich.1,google,google.1,"Bielik, František."
0,uc1.$b396528,deny,ic,1425,NaN,UC,.b139562850,103154,333110706,NaN,...,1970.0,enk,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Hughes, Glyn, 1935-"
1,uc1.$b392882,deny,ic,3229,NaN,UC,.b103086195,220000,NaN,NaN,...,1968.0,enk,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Perlès, Alfred."
2,uc1.$b396493,deny,ic,3413,NaN,UC,.b13956696x,235518,NaN,NaN,...,1971.0,ce,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Gooneratne, Yasmine, 1935-"
3,mdp.39015000628464,deny,ic,3838,NaN,MIU,000003838,265046,NaN,NaN,...,1951.0,ilu,eng,BK,MIU,umich,umich,google,google,"Durkheim, Emile, 1858-1917."
4,mdp.39015047487577,deny,ic,4878,v.7,MIU,000004878,337244,NaN,NaN,...,1967.0,nyu,eng,BK,MIU,umich,umich,google,google,"Tannenbaum, Samuel Aaron, 1874?-1948."


In [58]:
df2.shape[1]

26

In [59]:
# Similarly, we rename the columns and select columns
# a "readme" file for Hathifiles can be found at https://www.hathitrust.org/hathifiles_description
df2.columns=['htid','access','rights', 'ht_bib_key', 'description','source','source_bib_num','oclc_num', 
             'isbn','issn','iccn','title','imprint','rights_reason_code','rights_timestamp','us_gov_doc_flag',
             'rights_date_used','pub_place','lang','bib_fmt','collection_code', 'content_provider_code',
             'responsible_entity_code','digitization_agent_code','access_profile_code','author']

In [60]:
df2.head()

,htid,access,rights,ht_bib_key,description,source,source_bib_num,oclc_num,isbn,issn,...,rights_date_used,pub_place,lang,bib_fmt,collection_code,content_provider_code,responsible_entity_code,digitization_agent_code,access_profile_code,author
0,uc1.$b396528,deny,ic,1425,NaN,UC,.b139562850,103154,333110706,NaN,...,1970.0,enk,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Hughes, Glyn, 1935-"
1,uc1.$b392882,deny,ic,3229,NaN,UC,.b103086195,220000,NaN,NaN,...,1968.0,enk,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Perlès, Alfred."
2,uc1.$b396493,deny,ic,3413,NaN,UC,.b13956696x,235518,NaN,NaN,...,1971.0,ce,eng,BK,NRLF,universityofcalifornia,universityofcalifornia,google,google,"Gooneratne, Yasmine, 1935-"
3,mdp.39015000628464,deny,ic,3838,NaN,MIU,000003838,265046,NaN,NaN,...,1951.0,ilu,eng,BK,MIU,umich,umich,google,google,"Durkheim, Emile, 1858-1917."
4,mdp.39015047487577,deny,ic,4878,v.7,MIU,000004878,337244,NaN,NaN,...,1967.0,nyu,eng,BK,MIU,umich,umich,google,google,"Tannenbaum, Samuel Aaron, 1874?-1948."


In [61]:
df2 = df2[['title','author']]
df2.head()

,title,author
0,"Neighbours: poems, 1965-69.","Hughes, Glyn, 1935-"
1,Scenes from a floating life: excerpts from an ...,"Perlès, Alfred."
2,"Word, bird, motif.","Gooneratne, Yasmine, 1935-"
3,"Suicide, a study in sociology: translated by J...","Durkheim, Emile, 1858-1917."
4,"Elizabethan bibliographies, by Samuel A. and D...","Tannenbaum, Samuel Aaron, 1874?-1948."


## Accurate Matching
find the pairs where the two author strings and the two title strings are exactly the same

In [62]:
df_output=pd.merge(df, df2, on=['title','author']).drop_duplicates()
df_output.to_csv(r'accurate_match.csv')

In [63]:
print(df_output)

Empty DataFrame
Columns: [title, author]
Index: []


In [65]:
# No match found

## Fuzzy Matching
We only use the first 30 characters in a title because there are many redundant"volumn" information

In [91]:
def fuzzyMatching(path):
    df=pd.read_csv('booksummaries.txt',sep='\t',nrows=1000)
    df.columns = ['id','Freebase ID','title','author','ublication date','Book genres','Plot summary']
    df = df[['id','title','author']]
    df.name = str(path)
    #print(df.head())
    authorratio = []
    titleratio = []
    sourceauthor = []
    htrcauthor = []
    sourcetitle = []
    htrctitle = []
    sourceIDlist= []
    htrcDocid = []
    #count df items in the loop
    counter = 0
    #count df2 items in the loop
    counter2 = 0
    # count number of output files
    df2=pd.read_csv('hathi_upd_20190801.txt',sep='\t',nrows=100)
    df2.columns=['htid','access','rights', 'ht_bib_key', 'description','source','source_bib_num','oclc_num', 
             'isbn','issn','iccn','title','imprint','rights_reason_code','rights_timestamp','us_gov_doc_flag',
             'rights_date_used','pub_place','lang','bib_fmt','collection_code', 'content_provider_code',
             'responsible_entity_code','digitization_agent_code','access_profile_code','author']
    df2 = df2[['htid','title','author']]
    titlelist = df['title'].values
    authorlist = df["author"].values
    sourceidlist=df['id'].values
    for i in range(len(df)):
        counter = counter + 1
        author = authorlist[i]
        sourceid = sourceidlist[i]
        title = str(titlelist[i])[0:30]
        authorlist2 = df2["author"].values
        titlelist2 = df2['title'].values
        htrcgidlist = df2['htid'].values
        for j in range(len(df2)):
            counter2 = counter2 + 1
            author2 = authorlist2[j]
            title2 = str(titlelist2[j])[0:30]
            htrcid2 = htrcgidlist[j]
            aRatio = fuzz.ratio(str(author).lower(), str(author2).lower())
            tRatio = fuzz.ratio(str(title).lower(), str(title2).lower())
            if aRatio > 60 and tRatio > 70:
                sourceIDlist.append(sourceid)
                htrcDocid.append(htrcid2)
                authorratio.append(aRatio)
                sourceauthor.append(author)
                htrcauthor.append(author2)
                titleratio.append(tRatio)
                sourcetitle.append(title)
                htrctitle.append(title2)
            else:
                pass
    print(len(sourceauthor), len(htrctitle))
    #print(counter,counter2)
    df_output = pd.DataFrame(list(zip(authorratio, titleratio, sourceauthor, htrcauthor, sourcetitle, htrctitle, sourceIDlist, 
            htrcDocid)), columns=['authorratio', 'titleratio', 'sourceauthor', 'htrcauthor', 'sourcetitle', 'htrctitle',
                                      'sourceId', 'htrcDocid'])
    fileName=df.name
    df_output.to_csv('fuzzyMatchingResult.csv', index=False)
    print('done')

In [92]:
#this is a simple test, we did not find any pairs.
path='booksummaries.txt'
fuzzyMatching(path)

0 0
done


In [ ]:
# this is an exmple of some successful fuzzy matching cases using metadata datasets of British Periodicals and HTRC
# first two rows represents the reliablity(100%) of the matching

In [4]:
demo=pd.read_csv('example_bpo_htrc.csv')
demo

,authorratio,titleratio,sourceauthor,htrcauthor,sourcetitle,htrctitle,sourceId,htrcDocid
0,62,84,Pope Alexander,Nobile Achilles Alexander,Miscellaneous poems,Miscellaneous works,5502314,uc1.$b29217
1,62,84,Pope Alexander,Nobile Achilles Alexander,Miscellaneous poems,Miscellaneous works,5681605,uc1.$b29217
2,62,84,Pope Alexander,Nobile Achilles Alexander,Miscellaneous poems,Miscellaneous works,5502468,uc1.$b29217
3,62,84,Pope Alexander,Nobile Achilles Alexander,Miscellaneous poems,Miscellaneous works,5500474,uc1.$b29217
4,100,100,MDonogh Felix,MDonogh Felix,The hermit abroad,The hermit abroad,8616338,njp.32101015324427
5,100,100,St Leger Barry,St Leger Barry,Some account of the life of th,Some account of the life of th,8208057,uc2.ark+=13960=t2w37rs3z
6,100,100,Morier James Justinian,Morier James Justinian,The adventures of Hajji Baba o,The adventures of Hajji Baba o,2521932,nyp.33433074915038
7,100,100,Pope Alexander,Pope Alexander,Miscellaneous poems,Miscellaneous poems,5513463,nyp.33433076086945
8,100,100,Pope Alexander,Pope Alexander,Miscellaneous poems,Miscellaneous poems,5512606,nyp.33433076086945
9,100,78,Kelty Mary Ann,Kelty Mary Ann,The story of Isabel,The story of Isabel by the aut,5512486,njp.32101068971082
